In [ ]:
import pandas as pd

# Load the MSA (Metropolitan Statistical Area) data by ZIP code from CSV
df_msa = pd.read_csv("data/msa-by-zip.csv")

# Filter out ZIP codes less than 601 (assuming these are invalid or out of scope)
df_msa = df_msa[df_msa["ZIP CODE"] >= 601]

# Convert ZIP codes to string format and pad with leading zeros to ensure 5-digit ZIP codes
df_msa["ZIP CODE"] = df_msa["ZIP CODE"].astype(str).str.zfill(5)

# Clean column names by converting to lowercase and replacing spaces with underscores
df_msa.columns = [col.lower().replace(" ", "_") for col in df_msa.columns]

# Create a subset dataframe with only the ZIP code and MSA name columns
df_subset = df_msa[["zip_code", "msa_name"]]

# Display the first few rows of the processed dataframe
df_msa.head()

,zip_code,state,msa_no.,gpci,gpci.1,gpci.2,county_no.,msa_name,unnamed:_8
8,00601,PR,99072.0,1.0,0.845,0.249,1.0,PR NONMETROPOLITAN AREA,NaN
9,00602,PR,10380.0,1.0,0.845,0.249,3.0,"Aguadilla-Isabela-San Sebastian, PR MSA",NaN
10,00603,PR,10380.0,1.0,0.845,0.249,5.0,"Aguadilla-Isabela-San Sebastian, PR MSA",NaN
11,00604,PR,10380.0,1.0,0.845,0.249,5.0,"Aguadilla-Isabela-San Sebastian, PR MSA",NaN
12,00605,PR,10380.0,1.0,0.845,0.249,5.0,"Aguadilla-Isabela-San Sebastian, PR MSA",NaN


In [ ]:
def clean_demo_data(df, df_subset, year):
    # Use the first row of the dataframe as the new header (column names)
    df.columns = df.iloc[0]
    # Drop the original first row since it's now the header, and reset the index
    df = df[1:].reset_index(drop=True)

    # Extract 5-digit ZIP codes from the 'Geographic Area Name' column using regex
    # Looks for the pattern "ZCTA5 <5-digit-zip>"
    df["zip_code"] = df["Geographic Area Name"].str.extract(r"ZCTA5 (\d{5})")

    # Identify column(s) related to White population percentage in the ACS dataset
    white_percent_col = [
        col for col in df.columns 
        if isinstance(col, str) and (
            "Percent!!RACE!!One race!!White" in col 
            or "Percent Estimate!!RACE!!Total population!!One race!!White" in col
            or "Percent!!RACE!!Total population!!One race!!White" in col)
    ]
    
    # Keep only the 'zip_code' and white percentage columns for further processing
    df = df[["zip_code"] + white_percent_col]

    # Merge demographic data with MSA information by ZIP code from df_subset
    df_merged = pd.merge(df, df_subset, on="zip_code", how="left")

    # Remove rows where MSA is labeled as NONMETROPOLITAN
    df_merged = df_merged[~df_merged["msa_name"].str.contains("NONMETROPOLITAN", na=False)]
    # Drop rows that do not have any MSA information after merge
    df_merged = df_merged.dropna(subset=["msa_name"])

    # Add the year column to the dataframe for reference
    df_merged["year"] = year

    # If white percentage column exists, convert white percentage to nonwhite percentage
    if white_percent_col:
        col = white_percent_col[0]
        # Convert the white percentage column to numeric and subtract from 100 to get nonwhite %
        df_merged["percent_nonwhite"] = 100.0 - pd.to_numeric(df_merged[col], errors="coerce")
    
    # Drop rows with missing values in 'percent_nonwhite' column (e.g., conversion errors)
    df_merged = df_merged.dropna(subset=["percent_nonwhite"])

    # Calculate the median nonwhite percentage per MSA region and create a new column with this value
    df_merged["region_nonwhite"] = df_merged.groupby("msa_name")["percent_nonwhite"].transform("median")

    # Standardize column names by converting to lowercase and replacing spaces with underscores
    df_merged.columns = [col.lower().replace(" ", "_") for col in df_merged.columns]

    # Return the cleaned and processed dataframe
    return df_merged


In [70]:
#Test on 2011 acs demo data
df_2011 = pd.read_csv("demodata/2011_demo_data.csv")
df_2011_cleaned = clean_demo_data(df_2011, df_subset, 2011)
df_2011_cleaned.head()

/var/folders/mx/cv_f44qs0m57p0lzjb0p91kh0000gn/T/ipykernel_53798/4172995476.py:2: DtypeWarning: Columns (2,3,4,6,7,10,11,14,15,18,19,22,23,26,27,30,31,34,35,38,39,42,43,46,47,50,51,54,55,58,59,62,63,70,71,74,75,78,79,82,83,86,87,88,90,91,94,95,98,99,100,102,103,106,107,110,111,112,114,115,118,119,122,123,126,127,130,131,134,135,138,139,142,143,146,147,150,151,154,155,158,159,162,163,166,167,170,171,174,175,178,179,182,183,186,187,190,191,194,195,198,199,202,203,206,207,210,211,214,215,218,219,222,223,226,227,230,231,232,234,235,238,239,242,243,246,247,250,251,254,255,258,259,260,262,263,266,267,270,271,274,275,278,279,282,283,286,287,290,291,294,295,298,299,302,303,306,307,310,311,314,315,318,319,322,323) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2011 = pd.read_csv("demodata/2011_demo_data.csv")


,zip_code,percent!!race!!one_race!!white,msa_name,year,percent_nonwhite,region_nonwhite
1,00602,85.9,"Aguadilla-Isabela-San Sebastian, PR MSA",2011,14.1,12.75
2,00603,89.2,"Aguadilla-Isabela-San Sebastian, PR MSA",2011,10.8,12.75
4,00610,91.7,"Aguadilla-Isabela-San Sebastian, PR MSA",2011,8.3,12.75
5,00612,82.8,"San Juan-Caguas-Guaynabo, PR MSA",2011,17.2,30.90
6,00616,92.4,"San Juan-Caguas-Guaynabo, PR MSA",2011,7.6,30.90


In [ ]:
data = []  # Initialize an empty list to collect cleaned dataframes

# Loop through each year from 2012 to 2023 (inclusive)
for year in range(2012, 2024):
    print(f"Starting year: {year}")  # Print progress message for each year
    
    # Construct the file path for the demographic data CSV of the given year
    file_name = "demodata/" + str(year) + "_demo_data.csv"
    
    # Load the raw demographic data for the current year
    df_demo = pd.read_csv(file_name)
    
    # Clean and process the raw data using the 'clean_demo_data' function
    # Pass the dataframe, MSA subset dataframe, and current year as arguments
    df_demo_cleaned = clean_demo_data(df_demo, df_subset, year)
    
    # Append the cleaned dataframe to the list
    data.append(df_demo_cleaned)

# Combine all yearly cleaned dataframes into one large dataframe, resetting the index
demo_2011_2023 = pd.concat(data, ignore_index=True)

# Save the combined cleaned demographic data for all years to a CSV file
demo_2011_2023.to_csv("demo_data_2012-2023.csv", index=False)


Starting year: 2012


/var/folders/mx/cv_f44qs0m57p0lzjb0p91kh0000gn/T/ipykernel_53798/3321356089.py:9: DtypeWarning: Columns (2,3,4,6,7,10,11,14,15,18,19,22,23,26,27,30,31,34,35,38,39,42,43,46,47,50,51,54,55,58,59,62,63,70,71,74,75,78,79,82,83,86,87,88,90,91,94,95,98,99,100,102,103,106,107,110,111,112,114,115,118,119,122,123,126,127,130,131,134,135,138,139,142,143,146,147,150,151,154,155,158,159,162,163,166,167,170,171,174,175,178,179,182,183,186,187,190,191,194,195,198,199,202,203,206,207,210,211,214,215,218,219,222,223,226,227,230,231,232,234,235,238,239,242,243,246,247,250,251,254,255,258,259,260,262,263,266,267,270,271,274,275,278,279,282,283,286,287,290,291,294,295,298,299,302,303,306,307,310,311,314,315,318,319,322,323) have mixed types. Specify dtype option on import or set low_memory=False.
  df_demo = pd.read_csv(file_name)


Starting year: 2013


/var/folders/mx/cv_f44qs0m57p0lzjb0p91kh0000gn/T/ipykernel_53798/3321356089.py:9: DtypeWarning: Columns (2,3,4,6,7,10,11,14,15,18,19,22,23,26,27,30,31,34,35,38,39,42,43,46,47,50,51,54,55,58,59,62,63,70,71,74,75,78,79,82,83,86,87,88,90,91,94,95,98,99,100,102,103,106,107,110,111,112,114,115,118,119,122,123,126,127,130,131,134,135,138,139,142,143,146,147,150,151,154,155,158,159,162,163,166,167,170,171,174,175,178,179,182,183,186,187,190,191,194,195,198,199,202,203,206,207,210,211,214,215,218,219,222,223,226,227,230,231,232,234,235,238,239,242,243,246,247,250,251,254,255,258,259,260,262,263,266,267,270,271,274,275,278,279,282,283,286,287,290,291,294,295,298,299,302,303,306,307,310,311,314,315,318,319,322,323) have mixed types. Specify dtype option on import or set low_memory=False.
  df_demo = pd.read_csv(file_name)


Starting year: 2014


...

/var/folders/mx/cv_f44qs0m57p0lzjb0p91kh0000gn/T/ipykernel_53798/3321356089.py:9: DtypeWarning: Columns (2,3,4,5,6,7,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,58,59,60,61,62,63,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,228,236,244,304,318,352) have mixed types. Specify dtype option on import or set low_memory=False.
  df_demo = pd.read_csv(file_name)


Starting year: 2022


/var/folders/mx/cv_f44qs0m57p0lzjb0p91kh0000gn/T/ipykernel_53798/3321356089.py:9: DtypeWarning: Columns (2,3,4,5,6,7,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,58,59,60,61,62,63,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,232,240,248,312,326,360) have mixed types. Specify dtype option on import or set low_memory=False.
  df_demo = pd.read_csv(file_name)


Starting year: 2023


/var/folders/mx/cv_f44qs0m57p0lzjb0p91kh0000gn/T/ipykernel_53798/3321356089.py:9: DtypeWarning: Columns (2,3,4,5,6,7,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,58,59,60,61,62,63,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,238,246,254,324,338,372) have mixed types. Specify dtype option on import or set low_memory=False.
  df_demo = pd.read_csv(file_name)
